In [2]:
import pickle
import torch                                            # фреймворк для машинного обучения и нейросетей
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression

In [2]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for huggingface-hub<1.0,>=0.15.1 from https://files.pythonhosted.org/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.7 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for safetensors>=0.3.1 from https://files.pythonhosted.org/packages/9c/ee/de72fa290bef49b22aacdf112e70f40d3b1beb526dd65e94e0f540d192f0/safetensors-0.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency informa

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

In [4]:
loaded_model = pickle.load(open('veidlink/aux/LogReg.pickle', "rb"))

/home/veidlink/miniforge3/envs/NN/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [39]:
# Функция для предсказания класса строки
def predict_single_string(text: str,
                          bert_model:  BertModel,
                          logreg_model: LogisticRegression
) -> str:

  with torch.no_grad():
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    vector = output[0][:,0,:]
    pred0 = loaded_model.predict_proba(vector)[0][0]
    pred1 = loaded_model.predict_proba(vector)[0][1]
  if pred0 > pred1:
    result = 'Негативный отзыв'
    return  f'{result}, уверенность: {pred0:.2f}'
  else:
    result = 'Позитивный отзыв'
    return  f'{result}, уверенность: {pred1:.2f}'

### Хорошо работает на длинных отзывах, хуже на маленьких

In [35]:
text1 = 'Прекрасная больница'
text2 = 'Ужасная больница'
text3 = 'Обратилась к заведующей для постановки на учет по беременности. Я - россиянка, имею полис по г. Москве, регистрацию до 2016 г., мне 40 лет, и я работаю главным бухгалтером, а в ответ получила такую дозу хамства (ее высказывания пропущу), что в результате этого визита попала в больницу. УДИВИТЕЛЬНО, что врач вообще способен на хамство, а уж тем более руководитель учреждения, очень хотелось бы чтобы руководство поликлиники №181 г. Москва, обратило внимание на проф. пригодность своих сотрудников'

In [36]:
predict_single_string(text1, model, loaded_model) # позитивный должен быть

'Позитивный отзыв, уверенность: 0.80'

In [37]:
predict_single_string(text2, model, loaded_model) # негативный должен быть

'Позитивный отзыв, уверенность: 0.72'

In [38]:
predict_single_string(text3, model, loaded_model) # негативный должен быть

'Негативный отзыв, уверенность: 0.76'

F1 Macro на Train - *0.83* \
F1 Macro на Test - *0.82*